In [ ]:
import os
import sys
import pymssql
import locale

sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np


from dotenv import load_dotenv
from num2words import num2words
from docxtpl import DocxTemplate
from docx2pdf import convert
from utils_ccd import get_connection, generate_pdf_office

load_dotenv()
conn = get_connection()
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

c:\Users\05911205424\Documents\Dev\ccd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# mês atual, ano atual
from datetime import datetime
now = datetime.now()
mes_atual = now.month
ano_atual = now.year


#somar valores das multas

In [4]:
descontos_sql = f''' 
SELECT 
    pro.numero_processo,
    pro.ano_processo,
    pro.assunto,
    pm.Descricao as descricao,
    CONCAT(gp.Nome, ' (CPF: ', gp.Documento, ')') as nome,
    processo.dbo.fn_Exe_RetornaValorAtualizado(ed.IdDebito) as valor_multa,
    orgao_info.nome_orgao,
    ed.CodigoStatusDivida as status_divida
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
    ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm 
    ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed 
    ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
    ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp 
    ON gp.IdPessoa = edp.IDPessoa
OUTER APPLY (
    SELECT TOP 1 vspfr.nome_orgao
    FROM BdDIP.dbo.vwSiaiPessoalFolhaResumida vspfr
    WHERE vspfr.cpf = gp.Documento COLLATE SQL_Latin1_General_CP1_CI_AS
    AND vspfr.mes = {mes_atual - 1} AND vspfr.ano = {ano_atual}
    ORDER BY vspfr.ano DESC, vspfr.mes DESC
) AS orgao_info
WHERE 
    pro.setor_atual = 'CCD'
    AND pm.Descricao LIKE '%DESCONTO EM FOLHA - Implementar%'
    AND ed.CodigoTipoDebito NOT IN (1, 3)
'''
processos_descontos = pd.read_sql(descontos_sql, conn)

In [7]:
descontos_sql_nereu = f''' 
SELECT 
    pro.numero_processo,
    pro.ano_processo,
    pro.assunto,
    pm.Descricao as descricao,
    CONCAT(gp.Nome, ' (CPF: ', gp.Documento, ')') as nome,
    processo.dbo.fn_Exe_RetornaValorAtualizado(ed.IdDebito) as valor_multa,
    'INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE DO NORTE' as nome_orgao,
    ed.CodigoStatusDivida as status_divida
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
    ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm 
    ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed 
    ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
    ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp 
    ON gp.IdPessoa = edp.IDPessoa
WHERE 
    pro.setor_atual = 'CCD'
    AND CONCAT(pro.numero_processo, '/', pro.ano_processo) IN ('001361/2022',
'003654/2022',
'002748/2022',
'000133/2022',
'002739/2022',
'003061/2022',
'000152/2022',
'000129/2022',
'003054/2022',
'003010/2022',
'003010/2022',
'000112/2022',
'000116/2022',
'000115/2022',
'000111/2022',
'000125/2022',
'000128/2022',
'000157/2022',
'000123/2022',
'001425/2022',
'002716/2022',
'001365/2022',
'001423/2022',
'001362/2022',
'001366/2022',
'000126/2022',
'000131/2022',
'002682/2022',
'003006/2022',
'000119/2022',
'002062/2024',
'002038/2024',
'000470/2024',
'000142/2023',
'000101/2022',
'003078/2022',
'003023/2022',
'003668/2022',
'003690/2022',
'003709/2022',
'003687/2022',
'000421/2024',
'003659/2022',
'003667/2022',
'003663/2022',
'003064/2022',
'000088/2023',
'003686/2022',
'000157/2023',
'003701/2022',
'003263/2023',
'003700/2022',
'000480/2024',
'012970/2017'
) 
    AND ed.CodigoTipoDebito NOT IN (1, 3)
'''
processos_descontos = pd.read_sql(descontos_sql_nereu, conn)

In [8]:
processos_descontos

,numero_processo,ano_processo,assunto,descricao,nome,valor_multa,nome_orgao,status_divida
0,012970,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,Maria das Graças Fernandes Costa da Motta (CPF...,841.0762,INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1
1,012970,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),9916.7749,INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,15
2,012970,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),31128.6120,INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,15
3,000101,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),14440.0244,INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,15
4,000101,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),8985.1035,INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1
...,...,...,...,...,...,...,...,...
60,000421,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),1377.4151,INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1
61,000470,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),10873.7846,INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1
62,000480,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),1194.0324,INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1
63,002038,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),12074.7331,INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1


In [9]:
processos_descontos = processos_descontos[~processos_descontos.nome_orgao.isna()]

In [10]:
def criar_valor_extenso(x):
        if x and x > 1:
            f = lambda c: num2words(int(c), lang='pt_BR')
            return ' reais e '.join([f(y) for y in str('{:.2f}'.format(x)).split('.')]) + ' centavos'
        else:
            return ''

def converter_valor(x):
    if x:
        return locale.currency(x, grouping=True) + ' (' + criar_valor_extenso(x) + ')'
    else:
        return 'R$ 0,00 (zero)'

In [11]:
processos_descontos.fillna(0, inplace=True)

In [12]:
processos_descontos = processos_descontos.groupby(['numero_processo', 'ano_processo', 'assunto', 'descricao', 'nome'
    ,'nome_orgao', 'status_divida']).valor_multa.sum().reset_index() 

In [13]:
processos_descontos['processo'] = processos_descontos['numero_processo'] + '/' + processos_descontos['ano_processo'].astype(str)
processos_descontos['valor'] = processos_descontos['valor_multa'].apply(converter_valor)  

In [14]:
processos_descontos.columns

Index(['numero_processo', 'ano_processo', 'assunto', 'descricao', 'nome',
       'nome_orgao', 'status_divida', 'valor_multa', 'processo', 'valor'],
      dtype='object')

In [15]:
descontos_aberto = processos_descontos[processos_descontos.status_divida == 1]

In [16]:
descontos_aberto

,numero_processo,ano_processo,assunto,descricao,nome,nome_orgao,status_divida,valor_multa,processo,valor
0,000088,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,1348.2135,000088/2023,"R$ 1.348,21 (mil, trezentos e quarenta e oito ..."
1,000101,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,8985.1035,000101/2022,"R$ 8.985,10 (oito mil, novecentos e oitenta e ..."
3,000111,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,14305.5525,000111/2022,"R$ 14.305,55 (catorze mil, trezentos e cinco r..."
4,000112,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,14305.5525,000112/2022,"R$ 14.305,55 (catorze mil, trezentos e cinco r..."
5,000115,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,14305.5525,000115/2022,"R$ 14.305,55 (catorze mil, trezentos e cinco r..."
6,000116,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,14305.5525,000116/2022,"R$ 14.305,55 (catorze mil, trezentos e cinco r..."
7,000119,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,12458.1314,000119/2022,"R$ 12.458,13 (doze mil, quatrocentos e cinquen..."
8,000123,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,13952.6624,000123/2022,"R$ 13.952,66 (treze mil, novecentos e cinquent..."
9,000125,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,14305.5525,000125/2022,"R$ 14.305,55 (catorze mil, trezentos e cinco r..."
10,000126,2022,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar Nereu,NEREU BATISTA LINHARES (CPF: 13006444434),INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE ...,1,13333.8339,000126/2022,"R$ 13.333,83 (treze mil, trezentos e trinta e ..."


In [17]:
for desconto in descontos_aberto.itertuples():
    doc = DocxTemplate("templates/desconto_folha.docx")
    context = {
        'processo': desconto.processo,
        'assunto': desconto.assunto,
        'nome': desconto.nome,
        'orgao': desconto.nome_orgao,
        'valor': desconto.valor
    }
    nome = desconto.nome.split()[0].lower()
    processo_underline = desconto.processo.replace('/', '_')

    doc.render(context)
    d = f"saidas/desconto_folha/{processo_underline}_{nome}.docx"
    doc.save(d)
    generate_pdf_office(d, "saidas/desconto_folha/")
    print(f"Gerado arquivo para o processo {desconto.processo}")

Gerado arquivo para o processo 000088/2023
Gerado arquivo para o processo 000101/2022
Gerado arquivo para o processo 000111/2022
Gerado arquivo para o processo 000112/2022
Gerado arquivo para o processo 000115/2022
Gerado arquivo para o processo 000116/2022
Gerado arquivo para o processo 000119/2022
Gerado arquivo para o processo 000123/2022
Gerado arquivo para o processo 000125/2022
Gerado arquivo para o processo 000126/2022
Gerado arquivo para o processo 000128/2022
Gerado arquivo para o processo 000129/2022
Gerado arquivo para o processo 000131/2022
Gerado arquivo para o processo 000133/2022
Gerado arquivo para o processo 000142/2023
Gerado arquivo para o processo 000152/2022
Gerado arquivo para o processo 000157/2022
Gerado arquivo para o processo 000157/2023
Gerado arquivo para o processo 000421/2024
Gerado arquivo para o processo 000470/2024
Gerado arquivo para o processo 000480/2024
Gerado arquivo para o processo 001361/2022
Gerado arquivo para o processo 001362/2022
Gerado arqu